In [32]:
import json
import os
import glob
import pandas as pd
from tqdm import tqdm
import datetime
import numpy as np
import time

In [20]:
def return_master_df(tourny_prefix):
       
    os.chdir('/home/valesco/Datasets/PGA_Data/PGA_Dir/' + tourny_prefix)
    master_player_holes_df = pd.read_csv(tourny_prefix + '_player_holes.csv', low_memory = False)
    master_player_holes_df.fillna(value = np.nan, inplace = True)
    
    return master_player_holes_df


def myround(x, base):
    return (int(base * round(float(x) / base)))

def calc_sg(df, shot):  
    
    if df['lasers'] == 'No':
        return np.nan, np.nan, np.nan
    else:
        try:
            shot_type_str = 't_' + str(shot)
            if df[shot_type_str] in ['P', 'D']:
                return (np.nan, np.nan, np.nan)
            else:
                if shot == 1:
                    hole_id = df['hole_id']
                    try:
                        start_dist = course_holes['yard_from_tee_to_pin'].loc[course_holes['hole_id'] == hole_id].values[0] * 36
                    except:
                        return np.nan, np.nan, np.nan
                    start_loc = 'OTB'
                else:
                    prev_hole_dist = 'dist_left_to_hole_' + str(shot - 1)
                    prev_hole_loc = 'end_loc' + str(shot - 1)
                    prev_hole_shot_id = 't_' + str(shot - 1)
                    this_hole_loc = 'end_loc' + str(shot)
                    start_dist = df[prev_hole_dist]
                    this_loc = df[this_hole_loc]
                    if df[prev_hole_shot_id] == 'D' and df[prev_hole_loc] == 'OTB':
                        start_loc = 'OFW'
                    else:
                        start_loc = df[prev_hole_loc]

                shot_traveled_col = 'dist_of_shot_' + str(shot)
                shot_traveled = df[shot_traveled_col]

                end_loc_col = 'end_loc' + str(shot)
                end_loc = df[end_loc_col]


                end_dist_col = 'dist_left_to_hole_' + str(shot)
                end_dist = df[end_dist_col]

                penalty_str = 't_' + str(shot + 1)

                if df[penalty_str] == 'P':
                    penalty = 1
                    end_loc_col = 'end_loc' + str(shot + 2)
                    end_loc = df[end_loc_col]

                    end_dist_col = 'dist_left_to_hole_' + str(shot + 2)
                    end_dist = df[end_dist_col]

                else:
                    penalty = 0


                if location_dict[start_loc] == 'verify':
                    shot_text_col = 'shot_text_' + str(shot - 1)
                    shot_text = df[shot_text_col]

                    if 'fairway' in shot_text:
                        start_loc_sg_col = 'Fairway'
                    elif 'green' in shot_text:
                        start_loc_sg_col = 'Green'
                    elif 'bunker' in shot_text: 
                        start_loc_sg_col = 'Sand'
                    elif 'intermediate' in shot_text:
                        start_loc_sg_col = 'Intermediate'
                    else:
                        start_loc_sg_col = 'Rough'
                else:
                    start_loc_sg_col = location_dict[start_loc]

                if start_dist > 0:
                    if penalty == 0 and (shot_traveled / start_dist) < .55 and myround(start_dist / 36, 20) > 99 and shot != 1:
                        start_loc_sg_col = 'Recovery'

                if location_dict[end_loc] == 'verify':
                    end_text_col = 'shot_text_' + str(shot)
                    end_text = df[end_text_col]

                    if 'fairway' in end_text:
                        end_loc_sg_col = 'Fairway'
                    elif 'green' in end_text:
                        end_loc_sg_col = 'Green'
                    elif 'bunker' in end_text: 
                        end_loc_sg_col = 'Sand'
                    elif 'intermediate' in end_text:
                        end_loc_sg_col = 'Intermediate'
                    else:
                        end_loc_sg_col = 'Rough'
                else:
                    end_loc_sg_col = location_dict[end_loc]

                next_hole_start_dist = end_dist
                next_hole_shot_traveled_col = 'dist_left_to_hole_' + str(shot + 1)
                next_hole_shot_drop_col = 't_' + str(shot + 1)
                next_hole_shot_traveled = df[next_hole_shot_traveled_col]
                next_hole_shot_drop = df[next_hole_shot_drop_col]


                if next_hole_shot_traveled > 1 and next_hole_shot_drop not in ['P', 'D']:
                    if myround(next_hole_shot_traveled / 36, 20) > 100:
                        if (next_hole_start_dist / next_hole_shot_traveled) > .55 and myround(next_hole_start_dist / 36, 20) > 100:
                            end_loc_sg_col = 'Recovery'

                if np.isnan(start_dist) or np.isnan(end_dist): #or start_dist < 20:
                    return (np.nan, np.nan, np.nan)
                else:
                    if start_loc_sg_col == 'Green' or start_loc_sg_col == 'Fringe':
                        rounded_dist_start = myround(start_dist / 12, 1)
                        if rounded_dist_start < 2:
                            rounded_dist_start = 1
                        start_expected_strokes = strokes_gained[start_loc_sg_col].loc[strokes_gained['PuttDistinfeet'] == rounded_dist_start].values[0]
                    else:
                        rounded_dist_start = myround(start_dist / 36, 20)
                        if rounded_dist_start == 0:
                            rounded_dist_start = 20
                        if start_loc_sg_col == 'Hole':
                            start_expected_strokes = 0
                        else:
                            start_expected_strokes = strokes_gained[start_loc_sg_col].loc[strokes_gained['Distance'] == rounded_dist_start].values[0]

                    if end_loc_sg_col == 'Green' or end_loc_sg_col == 'Fringe':
                        rounded_dist_end = myround(end_dist / 12, 1)
                        if rounded_dist_end == 0:
                            rounded_dist_end = 1
                        end_expected_strokes = strokes_gained[end_loc_sg_col].loc[strokes_gained['PuttDistinfeet'] == rounded_dist_end].values[0]
                    elif end_loc_sg_col == 'Hole':
                        end_expected_strokes = 0
                    else:
                        rounded_dist_end = myround(end_dist / 36, 20)
                        if rounded_dist_end == 0:
                            rounded_dist_end = 20
                        end_expected_strokes = strokes_gained[end_loc_sg_col].loc[strokes_gained['Distance'] == rounded_dist_end].values[0]

                    total_sg = start_expected_strokes - end_expected_strokes - penalty - 1


                return total_sg, start_loc_sg_col, start_dist
        except:
            return np.nan, np.nan, np.nan

def create_sg_df(tourny_prefix):
    master_player_holes_df = return_master_df(tourny_prefix)
    
    master_player_holes_df = master_player_holes_df.merge(course_holes[['hole_id', 'par']], on = 'hole_id')
    
    col_ls = []
    for i in range(1,12):
        column_name = 'shot_' + str(i) + '_strokes_gained'
        col_ls.append(column_name)

    sg_df = pd.DataFrame(columns = col_ls)

    for i, col in enumerate(col_ls):
        sg_df[col] = master_player_holes_df.apply(calc_sg, shot = i + 1, axis = 1) 
        
    sg_final_df = pd.DataFrame()

    for i in range(1,12):
        hole = str(i)
        col_name_1 = 'shot_' + hole + '_sg'
        col_name_2 = 'shot_' + hole + '_loc'
        col_name_3 = 'shot_' + hole + '_start_dist'
        col_name_4 = 'shot_' + hole + '_strokes_gained'

        sg_final_df[[col_name_1, col_name_2, col_name_3]] = sg_df[col_name_4].apply(pd.Series)
    
    master_player_holes_df = pd.concat([master_player_holes_df, sg_final_df], axis = 1)
    
    master_player_holes_df.fillna('0', inplace = True)
    
    final_df = pd.DataFrame(columns = ['player_id', 'player_name', 'date_started', 'tournament_id', 'course_num', 'year', 
                                'round_num', 'hole_num','hole_id', 'par', 'stroke_num', 'stroke_gained', 'shot_loc',
                                'shot_dist'])
    count = 0
    for index, row in master_player_holes_df.iterrows():
        if row['lasers'] == 'Yes':
            player_id = row['player_id']
            full_name = row['first_name'] + ' ' + row['last_name']
            hole_id = row['hole_id']
            tournament_id = row['tournament_id']
            course_num = row['course_num']
            year = row['year']
            date = row['date_started']
            round_num = row['round_num']
            hole_num = row['hole_num']
            hole_par = row['par']

            for i in range(1,12):
                sg_col = 'shot_{}_sg'.format(i)
                sg_loc_col = 'shot_{}_loc'.format(i)
                sg_dist_col = 'shot_{}_start_dist'.format(i)
                stroke_num_col = 'stroke_num_{}'.format(i)

                sg = row[sg_col]
                sg_loc = row[sg_loc_col]
                sg_dist = row[sg_dist_col]
                stroke_num = row[stroke_num_col]

                if sg_loc != '0':
                    final_df.loc[count] = [player_id, full_name, date, tournament_id, course_num, year, round_num, 
                                       hole_num, hole_id, hole_par, stroke_num, sg, sg_loc, sg_dist]

                    count += 1

    return final_df

In [36]:
time.sleep(4000)


strokes_gained = pd.read_csv('/home/valesco/Datasets/PGA_Data/PGA_strokes_gained.csv', index_col = 0)

location_dict = {
    'OFW': 'Fairway',
    'OGR': 'Green',
    'ORO': 'Rough',
    'OIR': 'Intermediate',
    'OUK': 'verify',
    'ONA': 'verify',
    'OST': 'Sand',
    'OTH': 'verify',
    'OTB': 'Tee',
    'OCO': 'Fringe',
    'OGS': 'Sand',
    np.nan : np.nan,
    'OGB': 'verify',
    'ERF': 'Fairway',
    'ELF': 'Fairway',
    'hole': 'Hole',
    'ELR': 'Rough',
    'EG5': 'Sand',
    'EG2': 'Sand',
    'ERI': 'Intermediate',
    'ERR': 'Rough',
    'OTB': 'Tee',
    'EG3': 'Sand',
    'ELI': 'Intermediate',
    'EG8': 'Sand',
    'EG1': 'Sand',
    'OTO': 'Rough',
    'OGS': 'Sand',
    'EG4': 'Sand',
    'EG6': 'Sand',
    'EG7': 'Sand',
    'OCA': 'Rough',
    'ODO': 'Rough',
    'OLN': 'Rough',
    'ORK': 'Rough', 
    'OBO': 'Rough',
    'OBR': 'Rough',
    'OPT': 'Rough',
    'OWL': 'Rough',
    'OBD': 'Rough',
    'OWA': 'verify'}


for tourny_prefix in tqdm(tournaments):
    course_holes = pd.read_csv('/home/valesco/Datasets/PGA_Data/PGA_Dir/{}/{}_course_holes.csv'.format(tourny_prefix, 
                                                                                                    tourny_prefix))
    strokes_gained_df = create_sg_df(tourny_prefix)
    strokes_gained_df.to_csv('/home/valesco/Datasets/PGA_Data/PGA_Dir/{}/{}_strokes_gained.csv'.format(tourny_prefix,
                                        tourny_prefix), index = False)
    



100%|██████████| 1/1 [03:58<00:00, 238.53s/it]


In [35]:
tournaments = [
'006_2017'
]